#### 필수 목표

분석 흐름 개요
- 데이터 EDA
- 데이터 정제 및 데이터 전처리
- 데이터 시각화 및 분석
- 데이터 기반 인사이트 도출

1. [비즈니스 목표 세우기]

2. [사용된 데이터 소스 설명] 출처, 구성, 관측단위, 주요 변수를 소개
- 관측 단위와 기간
    - 관측 단위: Fitbit 사용자(30명) × 활동/수면/심박 데이터, 사용자-일(Day)
    - 기간: 2016년 3월 12일 ~ 2016년 5월 12일(62일)
- 주요 변수 소개: 하드/노말/소프트 유저, 활동날짜()

3. [EDA] 행/열 개수를 제시
- 로딩 직후와 정제 후를 구분해서 행·열 개수(shape)와 결측치 개수를 각각 보고하기

4. [EDA] 분석할 데이터의 컬럼 타입과 기술통계(min/median/mean/max, 결측치 수)를 제시

5. [전처리 과정] 결측치/이상치/전처리 처리 규칙을 수립·실행하고, 처리 이유를 작성

6. [주요 분석제시 및 시각화] 한개 혹은 여러개의 기준 컬럼을 두고 집계함수로 비교분석 후 시각화

7. [인사이트 제시 ]최소 1개 이상의 인사이트를 수치/그래프/그림/해석으로 제시

#### 심화 목표

1. EDA를 더 다양한 형식(산점도, 박스플롯  등)으로 나타내보기

2. 사용자별 평균 활동량(예: mean_steps)을 기준으로 상위 20% / 하위 20% 그룹을 만들어 주요 지표(걸음수, 칼로리, 비활동 시간 등)를 비교

3. 월별/분기별 평균 활동량을 계산해 장기 추세(증가·감소)와 계절성(예: 여름·겨울 차이)을 파악

4. 요일별·시간대별 평균 Steps(또는 활동 비율)를 히트맵으로 그려 피크 시간대를 파악하고, 주중 vs 주말 차이를 분석


In [11]:
import pandas as pd
df34 = pd.read_csv("../data/dailyActivity_merged34.csv")
df45 = pd.read_csv("../data/dailyActivity_merged45.csv")

df35 = pd.concat([df34, df45])
df = df35.copy()

df.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,3/25/2016,11004,7.11,7.11,0.0,2.57,0.46,4.07,0.0,33,12,205,804,1819
1,1503960366,3/26/2016,17609,11.55,11.55,0.0,6.92,0.73,3.91,0.0,89,17,274,588,2154
2,1503960366,3/27/2016,12736,8.53,8.53,0.0,4.66,0.16,3.71,0.0,56,5,268,605,1944
3,1503960366,3/28/2016,13231,8.93,8.93,0.0,3.19,0.79,4.95,0.0,39,20,224,1080,1932
4,1503960366,3/29/2016,12041,7.85,7.85,0.0,2.16,1.09,4.61,0.0,28,28,243,763,1886


In [ ]:
df.info()
#(1397, 15)

<class 'pandas.core.frame.DataFrame'>
Index: 1397 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        1397 non-null   int64  
 1   ActivityDate              1397 non-null   object 
 2   TotalSteps                1397 non-null   int64  
 3   TotalDistance             1397 non-null   float64
 4   TrackerDistance           1397 non-null   float64
 5   LoggedActivitiesDistance  1397 non-null   float64
 6   VeryActiveDistance        1397 non-null   float64
 7   ModeratelyActiveDistance  1397 non-null   float64
 8   LightActiveDistance       1397 non-null   float64
 9   SedentaryActiveDistance   1397 non-null   float64
 10  VeryActiveMinutes         1397 non-null   int64  
 11  FairlyActiveMinutes       1397 non-null   int64  
 12  LightlyActiveMinutes      1397 non-null   int64  
 13  SedentaryMinutes          1397 non-null   int64  
 14  Calories      

In [ ]:
# 전체 데이터 기술통계
df.describe()

In [ ]:
# 결측치 확인
df.isnull().sum()

Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
dtype: int64

In [ ]:
#ID 종류 확인
df['ActivityDate'].nunique()

In [ ]:
# ID별 데이터 개수 확인
df2['Id'].value_counts()

In [ ]:
# ID별 총 걸음 수 확인
df2.groupby('Id')['TotalSteps'].mean()

In [ ]:
# 날짜 타입 변환
df2["ActivityDate"] = pd.to_datetime(df2["ActivityDate"])

# 요일 변수 생성
df2["weekday"] = df2["ActivityDate"].dt.day_name()

# 활동강도 합산 지표 생성
df2["TotalActiveMinutes"] = (df2["VeryActiveMinutes"]
                            + df2["FairlyActiveMinutes"]
                            + df2["LightlyActiveMinutes"])

df2.head()


In [ ]:
# 미착용일 조건 정의
non_wear_cond = (
    (df2["TotalSteps"] == 0) &
    (df2["TotalDistance"] == 0) &
    (df2["SedentaryMinutes"] >= 1380)   # 하루 23시간 이상 앉아있음
)

# 미착용일 제외
clean_df2 = df2.loc[~non_wear_cond].copy()

시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. 사용자별 데이터 기록 일수(Compliance) 계산
# nunique()를 사용해 중복 없이 실제로 며칠이나 데이터가 들어왔는지 확인합니다.
engagement = df2.groupby('Id')['ActivityDate'].nunique().reset_index()
engagement.columns = ['Id', 'RecordingDays']

# 2. 수동 입력(LoggedActivitiesDistance) 활용 여부 확인
# 수동으로 거리를 입력한 기록이 한 번이라도 있는지 합산하여 확인합니다.
manual_sum = df.groupby('Id')['LoggedActivitiesDistance'].sum().reset_index()
engagement['IsManualUser'] = manual_sum['LoggedActivitiesDistance'] > 0

# 3. 참여도 그룹 분류 함수
def segment_engagement(row):
    if row['IsManualUser']:
        return '수동 기록 그룹'  # 직접 입력하는 적극적 사용자
    elif row['RecordingDays'] >= 25:
        return '성실 착용 그룹'  # 거의 매일 착용
    elif row['RecordingDays'] >= 10:
        return '보통 착용 그룹'  # 가끔 착용
    else:
        return '이탈 위험 그룹'  # 거의 착용 안 함 (데이터 신뢰도 낮음)

engagement['Group'] = engagement.apply(segment_engagement, axis=1)

# 4. 결과 출력
print(engagement[['Id', 'RecordingDays', 'Group']])

In [ ]:
# 요약된 데이터로 막대 그래프 그리기
engagement.set_index('Id')['RecordingDays'].sort_values().plot(kind='barh', color='skyblue')
plt.title('User Recording Compliance (Days)')
plt.xlabel('Number of Days Recorded')
plt.axvline(x=15, color='red', linestyle='--') # 기준선 (예: 15일)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 날짜 데이터 처리 (문자열인 경우)
df2['ActivityDate'] = pd.to_datetime(df2['ActivityDate'])
df2['DayOfWeek'] = df2['ActivityDate'].dt.day_name()

# 2. 요일 순서 정의 (월요일 ~ 일요일)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 3. 요일 컬럼을 '순서가 있는 범주형'으로 변환
df2['DayOfWeek'] = pd.Categorical(df2['DayOfWeek'], categories=day_order, ordered=True)

# 4. 요일별 평균 계산 (예: 걸음 수 기준)
weekly_trend = df2.groupby('DayOfWeek')['TotalActiveMinutes'].mean()

# 5. 선 그래프 그리기
plt.figure(figsize=(10, 5))
weekly_trend.plot(kind='line', marker='o', color='b', linewidth=2)

# 그래프 꾸미기
plt.title('Weekly Activity Trend (Steps)', fontsize=14)
plt.xlabel('Day of the Week')
plt.ylabel('Total active')
plt.xticks(range(7), day_order) # x축 라벨 강제 지정
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# 박스 플롯

# sns.boxplot(x="day", y="total_bill", data=df2)
# plt.show()